In [52]:
# Import relevant libraries
from scipy import stats
import scipy as sp
import pandas as pd
import numpy as np
import random
import math

import matplotlib.pyplot as plt
import seaborn as sns

# Set seed for any random operations
random.seed(1)

In [7]:
# Read in test dataset (TODO: test data being used is temporary. replace with final test data.)
test_data = pd.read_csv("test.csv")
print "Number of test points:", test_data.shape[0]

Number of test points: 5481


In [12]:
# Number of simulations we want
N = 1000

In [3]:
# Predicts if loan is defaulted by time t in years
def is_defaulted_by(loan, years):
    # GIVEN LAMBDA
    default_prob = # Hand-calculate or calculate from hazard model
    u = np.random.uniform(0, 1)
    return (u <= default_prob)

SyntaxError: invalid syntax (<ipython-input-3-aa393f9270e9>, line 4)

In [ ]:
# Calculates loss of a loan using loss distribution model
def calculate_loss(loan):
    # PLUG IN LOSS DISTRIBUTION MODEL HERE
    return 100000

In [ ]:
# Simulates total loss of 500 loans for a t year horizon
def simulate_total_loss(N, years):
    total_losses = []
    for n in range(0, N):
        sampled_loans = test_data.sample(500)
        cur_loss = 0.0
        
        for i in range(0, 500):
            cur_loan = sampled_loans[i:i+1]
            if is_defaulted_by(cur_loan, years=1):
                loan_loss = calculate_loss(cur_loan)
                cur_loss += loan_loss
        
        total_losses.append(cur_loss)
    
    return total_losses

In [ ]:
# Estimate for 1 year and plot histogram
total_1_year_losses = simulate_total_loss(N, 1)
sns.distplot(total_1_year_losses)

In [ ]:
# Repeat for 5 year and plot histogram
total_5_year_losses = simulate_total_loss(N, 5)
sns.distplot(total_5_year_losses)

In [71]:
# Calculate VaR
def get_VAR(total_losses, level=0.95):
    sorted_losses = sorted(total_losses)
    index = int((1-level) * 500)
    return sorted_losses[index]

In [72]:
# Calculate Average VaR
def get_avg_VAR(total_losses, level=0.95):
    sampled_vars = []
    cur_level = 1.00
    while cur_level >= level:
        print cur_level
        print sampled_vars
        sampled_vars.append(get_VAR(total_losses, cur_level))
        cur_level -= 0.01
    return np.mean(sampled_vars)

In [50]:
# Calculate confidence interval
def get_conf_interval(total_losses, level=0.95):
    n, min_max, mean, var, skew, kurt = stats.describe(total_losses)
    std = math.sqrt(var)
    return stats.norm.interval(level, loc=mean, scale=std)

In [ ]:
print "VaR of one year loss at 95% level:", get_VAR(total_1_year_losses, 0.95)
print "VaR of one year loss at 99% level:", get_VAR(total_1_year_losses, 0.99)
print "Average VaR of one year loss at 95% level:", get_avg_VAR(total_1_year_losses, level=0.95)
print "Average VaR of one year loss at 99% level:", get_avg_VAR(total_1_year_losses, level=0.99)
print "95% confidence interval:", get_conf_interval(total_1_year_losses, level=0.95)
print "99% confidence interval:", get_conf_interval(total_1_year_losses, level=0.99)

print ""

print "VaR of five year loss at 95% level:", get_VAR(total_5_year_losses, 0.95)
print "VaR of five year loss at 99% level:", get_VAR(total_5_year_losses, 0.99)
print "Average VaR of five year loss at 95% level:", get_avg_VAR(total_5_year_losses, level=0.95)
print "Average VaR of five year loss at 99% level:", get_avg_VAR(total_5_year_losses, level=0.99)
print "95% confidence interval:", get_conf_interval(total_5_year_losses, level=0.95)
print "99% confidence interval:", get_conf_interval(total_5_year_losses, level=0.99)

In [ ]:
# Find junior [5%, 15%] and senior tranches [15%, 100%]